In [ ]:
!nvidia-smi -L

GPU 0: Tesla K80 (UUID: GPU-c83b84c0-c745-9360-6367-d40600c62986)


In [47]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#SL_GCN_test_trial

In [1]:
from __future__ import print_function
import argparse
from argparse import ArgumentParser,Namespace
import os
import time
import numpy as np
import yaml
import pickle
from collections import OrderedDict
# torch
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
from tqdm import tqdm
import shutil
from torch.optim.lr_scheduler import ReduceLROnPlateau, MultiStepLR
import random
import inspect
import torch.backends.cudnn as cudnn
import torch.nn.functional as F

In [2]:
def init_seed(_):
    torch.cuda.manual_seed_all(1)
    torch.manual_seed(1)
    np.random.seed(1)
    random.seed(1)
    # torch.backends.cudnn.enabled = False
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

In [3]:
def import_class(name):
    components = name.split('.')
    mod = __import__(components[0])  # import return model
    for comp in components[1:]:
        mod = getattr(mod, comp)
    return mod

In [4]:
class Processor():
    """ 
        Processor for Skeleton-based Action Recgnition
    """

    def __init__(self, arg):

        arg.model_saved_name = f"/content/drive/.shortcut-targets-by-id/1a7cCN6IAhkm0gj53vnNz_41b1ca-FH6C/Thesis Folder/Code Implementation/Keypoint to GCN/Dataset/weights_HRNet_trial/Joint_test/"+ arg.Experiment_name
        arg.work_dir = f"/content/drive/.shortcut-targets-by-id/1a7cCN6IAhkm0gj53vnNz_41b1ca-FH6C/Thesis Folder/Code Implementation/Keypoint to GCN/CVPR21Chal-SLR/SL-GCN/work_dir/Personal_use/" + arg.Experiment_name
        self.arg = arg
        self.save_arg()
        if arg.phase == 'train':
            if not arg.train_feeder_args['debug']:
                if os.path.isdir(arg.model_saved_name):
                    print('log_dir: ', arg.model_saved_name, 'already exist')
                    answer = input('delete it? y/n:')
                    if answer == 'y':
                        shutil.rmtree(arg.model_saved_name)
                        print('Dir removed: ', arg.model_saved_name)
                        input(
                            'Refresh the website of tensorboard by pressing any keys')
                    else:
                        print('Dir not removed: ', arg.model_saved_name)

        self.global_step = 0
        self.load_model()
        self.load_optimizer()
        self.load_data()
        self.lr = self.arg.base_lr
        self.best_acc = 0

    def load_data(self):
        Feeder = import_class(self.arg.feeder)
        self.data_loader = dict()
        if self.arg.phase == 'train':
            self.data_loader['train'] = torch.utils.data.DataLoader(
                dataset=Feeder(**self.arg.train_feeder_args),
                batch_size=self.arg.batch_size,
                shuffle=True,
                num_workers=self.arg.num_worker,
                drop_last=True,
                worker_init_fn=init_seed)
        self.data_loader['test'] = torch.utils.data.DataLoader(
            dataset=Feeder(**self.arg.test_feeder_args),
            batch_size=self.arg.test_batch_size,
            shuffle=False,
            num_workers=self.arg.num_worker,
            drop_last=False,
            worker_init_fn=init_seed)

    def load_model(self):
        output_device = self.arg.device[0] if type(
            self.arg.device) is list else self.arg.device
        self.output_device = output_device
        Model = import_class(self.arg.model)
        shutil.copy2(inspect.getfile(Model), self.arg.work_dir)
        self.model = Model(**self.arg.model_args).cuda(output_device)
        # print(self.model)
        self.loss = nn.CrossEntropyLoss().cuda(output_device)
        # self.loss = LabelSmoothingCrossEntropy().cuda(output_device)

        if self.arg.weights:
            self.print_log('Load weights from {}.'.format(self.arg.weights))
            if '.pkl' in self.arg.weights:
                with open(self.arg.weights, 'r') as f:
                    weights = pickle.load(f)
            else:
                weights = torch.load(self.arg.weights)

            weights = OrderedDict(
                [[k.split('module.')[-1],
                  v.cuda(output_device)] for k, v in weights.items()])

            for w in self.arg.ignore_weights:
                if weights.pop(w, None) is not None:
                    self.print_log('Sucessfully Remove Weights: {}.'.format(w))
                else:
                    self.print_log('Can Not Remove Weights: {}.'.format(w))

            try:
                self.model.load_state_dict(weights)
            except:
                state = self.model.state_dict()
                diff = list(set(state.keys()).difference(set(weights.keys())))
                print('Can not find these weights:')
                for d in diff:
                    print('  ' + d)
                state.update(weights)
                self.model.load_state_dict(state)

        if type(self.arg.device) is list:
            if len(self.arg.device) > 1:
                self.model = nn.DataParallel(
                    self.model,
                    device_ids=self.arg.device,
                    output_device=output_device)

    def load_optimizer(self):
        if self.arg.optimizer == 'SGD':

            params_dict = dict(self.model.named_parameters())
            params = []

            for key, value in params_dict.items():
                decay_mult = 0.0 if 'bias' in key else 1.0

                lr_mult = 1.0
                weight_decay = 1e-4

                params += [{'params': value, 'lr': self.arg.base_lr, 'lr_mult': lr_mult,
                            'decay_mult': decay_mult, 'weight_decay': weight_decay}]

            self.optimizer = optim.SGD(
                params,
                momentum=0.9,
                nesterov=self.arg.nesterov)
        elif self.arg.optimizer == 'Adam':
            self.optimizer = optim.Adam(
                self.model.parameters(),
                lr=self.arg.base_lr,
                weight_decay=self.arg.weight_decay)
        else:
            raise ValueError()

        self.lr_scheduler = ReduceLROnPlateau(self.optimizer, mode='min', factor=0.1,
                                              patience=10, verbose=True,
                                              threshold=1e-4, threshold_mode='rel',
                                              cooldown=0)

    def save_arg(self):
        # save arg
        arg_dict = vars(self.arg)

        if not os.path.exists(self.arg.work_dir):
            os.makedirs(self.arg.work_dir)
            os.makedirs(self.arg.work_dir + '/eval_results')

        with open('{}/config.yaml'.format(self.arg.work_dir), 'w') as f:
            yaml.dump(arg_dict, f)

    def adjust_learning_rate(self, epoch):
        if self.arg.optimizer == 'SGD' or self.arg.optimizer == 'Adam':
            if epoch < self.arg.warm_up_epoch:
                lr = self.arg.base_lr * (epoch + 1) / self.arg.warm_up_epoch
            else:
                lr = self.arg.base_lr * (
                    0.1 ** np.sum(epoch >= np.array(self.arg.step)))
            for param_group in self.optimizer.param_groups:
                param_group['lr'] = lr
            return lr
        else:
            raise ValueError()

    def print_time(self):
        localtime = time.asctime(time.localtime(time.time()))
        self.print_log("Local current time :  " + localtime)

    def print_log(self, str, print_time=True):
        if print_time:
            localtime = time.asctime(time.localtime(time.time()))
            str = "[ " + localtime + ' ] ' + str
        print(str)
        if self.arg.print_log:
            with open('{}/log.txt'.format(self.arg.work_dir), 'a') as f:
                print(str, file=f)

    def record_time(self):
        self.cur_time = time.time()
        return self.cur_time

    def split_time(self):
        split_time = time.time() - self.cur_time
        self.record_time()
        return split_time

    def train(self, epoch, save_model=False):
        self.model.train()
        self.print_log('Training epoch: {}'.format(epoch + 1))
        loader = self.data_loader['train']
        self.adjust_learning_rate(epoch)
        loss_value = []
        self.record_time()
        timer = dict(dataloader=0.001, model=0.001, statistics=0.001)
        process = tqdm(loader)
        if epoch >= self.arg.only_train_epoch:
            print('only train part, require grad')
            for key, value in self.model.named_parameters():
                if 'DecoupleA' in key:
                    value.requires_grad = True
                    print(key + '-require grad')
        else:
            print('only train part, do not require grad')
            for key, value in self.model.named_parameters():
                if 'DecoupleA' in key:
                    value.requires_grad = False
                    print(key + '-not require grad')
        for batch_idx, (data, label, index) in enumerate(process):
            self.global_step += 1
            # get data
            data = Variable(data.float().cuda(
                self.output_device), requires_grad=False)
            label = Variable(label.long().cuda(
                self.output_device), requires_grad=False)
            timer['dataloader'] += self.split_time()

            # forward
            if epoch < 100:
                keep_prob = -(1 - self.arg.keep_rate) / 100 * epoch + 1.0
            else:
                keep_prob = self.arg.keep_rate
            output = self.model(data, keep_prob)

            if isinstance(output, tuple):
                output, l1 = output
                l1 = l1.mean()
            else:
                l1 = 0
            loss = self.loss(output, label) + l1

            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()
            loss_value.append(loss.data)
            timer['model'] += self.split_time()

            value, predict_label = torch.max(output.data, 1)
            acc = torch.mean((predict_label == label.data).float())

            self.lr = self.optimizer.param_groups[0]['lr']

            if self.global_step % self.arg.log_interval == 0:
                self.print_log(
                    '\tBatch({}/{}) done. Loss: {:.4f}  lr:{:.6f}'.format(
                        batch_idx, len(loader), loss.data, self.lr))
            timer['statistics'] += self.split_time()

        # statistics of time consumption and loss
        proportion = {
            k: '{:02d}%'.format(int(round(v * 100 / sum(timer.values()))))
            for k, v in timer.items()
        }

        state_dict = self.model.state_dict()
        weights = OrderedDict([[k.split('module.')[-1],
                                v.cpu()] for k, v in state_dict.items()])

        os.makedirs(os.path.dirname(f'/content/drive/.shortcut-targets-by-id/1a7cCN6IAhkm0gj53vnNz_41b1ca-FH6C/Thesis Folder/Code Implementation/Keypoint to GCN/Dataset/weights_HRNet_trial/Joint_test/'), exist_ok=True)
        torch.save(weights, self.arg.model_saved_name +
                   '-' + str(epoch) + '.pt')

    def eval(self, epoch, save_score=False, loader_name=['test'], wrong_file=None, result_file=None):
        if wrong_file is not None:
            f_w = open(wrong_file, 'w')
        if result_file is not None:
            f_r = open(result_file, 'w')
        self.model.eval()
        with torch.no_grad():
            self.print_log('Eval epoch: {}'.format(epoch + 1))
            for ln in loader_name:
                loss_value = []
                score_frag = []
                right_num_total = 0
                total_num = 0
                loss_total = 0
                step = 0
                process = tqdm(self.data_loader[ln])

                for batch_idx, (data, label, index) in enumerate(process):
                    data = Variable(
                        data.float().cuda(self.output_device),
                        requires_grad=False)
                    label = Variable(
                        label.long().cuda(self.output_device),
                        requires_grad=False)

                    with torch.no_grad():
                        output = self.model(data)

                    if isinstance(output, tuple):
                        output, l1 = output
                        l1 = l1.mean()
                    else:
                        l1 = 0
                    loss = self.loss(output, label)
                    score_frag.append(output.data.cpu().numpy())
                    loss_value.append(loss.data.cpu().numpy())

                    _, predict_label = torch.max(output.data, 1)
                    step += 1

                    if wrong_file is not None or result_file is not None:
                        predict = list(predict_label.cpu().numpy())
                        true = list(label.data.cpu().numpy())
                        for i, x in enumerate(predict):
                            if result_file is not None:
                                f_r.write(str(x) + ',' + str(true[i]) + '\n')
                            if x != true[i] and wrong_file is not None:
                                f_w.write(str(index[i]) + ',' +
                                        str(x) + ',' + str(true[i]) + '\n')
                score = np.concatenate(score_frag)

                if 'UCLA' in arg.Experiment_name:
                    self.data_loader[ln].dataset.sample_name = np.arange(
                        len(score))

                accuracy = self.data_loader[ln].dataset.top_k(score, 1)
                if accuracy > self.best_acc:
                    self.best_acc = accuracy
                    score_dict = dict(
                        zip(self.data_loader[ln].dataset.sample_name, score))

                    with open(f'/content/drive/.shortcut-targets-by-id/1a7cCN6IAhkm0gj53vnNz_41b1ca-FH6C/Thesis Folder/Code Implementation/Keypoint to GCN/CVPR21Chal-SLR/SL-GCN/work_dir/Personal_use/' + arg.Experiment_name + '/eval_results/best_acc' + '.pkl'.format(
                            epoch, accuracy), 'wb') as f:
                        pickle.dump(score_dict, f)

                print('Eval Accuracy: ', accuracy,
                    ' model: ', self.arg.model_saved_name)

                score_dict = dict(
                    zip(self.data_loader[ln].dataset.sample_name, score))
                self.print_log('\tMean {} loss of {} batches: {}.'.format(
                    ln, len(self.data_loader[ln]), np.mean(loss_value)))
                for k in self.arg.show_topk:
                    self.print_log('\tTop{}: {:.2f}%'.format(
                        k, 100 * self.data_loader[ln].dataset.top_k(score, k)))

                with open(f'/content/drive/.shortcut-targets-by-id/1a7cCN6IAhkm0gj53vnNz_41b1ca-FH6C/Thesis Folder/Code Implementation/Keypoint to GCN/CVPR21Chal-SLR/SL-GCN/work_dir/Personal_use/' + arg.Experiment_name + '/eval_results/epoch_' + str(epoch) + '_' + str(accuracy) + '.pkl'.format(
                        epoch, accuracy), 'wb') as f:
                    pickle.dump(score_dict, f)
        return np.mean(loss_value)
    def start(self):
        if self.arg.phase == 'train':
            self.print_log('Parameters:\n{}\n'.format(str(vars(self.arg))))
            self.global_step = self.arg.start_epoch * \
                len(self.data_loader['train']) / self.arg.batch_size
            for epoch in range(self.arg.start_epoch, self.arg.num_epoch):
                save_model = ((epoch + 1) % self.arg.save_interval == 0) or (
                    epoch + 1 == self.arg.num_epoch)

                self.train(epoch, save_model=save_model)

                val_loss = self.eval(
                    epoch,
                    save_score=self.arg.save_score,
                    loader_name=['test'])

                # self.lr_scheduler.step(val_loss)

            print('best accuracy: ', self.best_acc,
                  ' model_name: ', self.arg.model_saved_name)

        elif self.arg.phase == 'test':
            if not self.arg.test_feeder_args['debug']:
                wf = self.arg.model_saved_name + '_wrong.txt'
                rf = self.arg.model_saved_name + '_right.txt'
            else:
                wf = rf = None
            if self.arg.weights is None:
                raise ValueError('Please appoint --weights.')
            self.arg.print_log = False
            self.print_log('Model:   {}.'.format(self.arg.model))
            self.print_log('Weights: {}.'.format(self.arg.weights))
            self.eval(epoch=self.arg.start_epoch, save_score=self.arg.save_score,
                      loader_name=['test'], wrong_file=wf, result_file=rf)
            self.print_log('Done.\n')

In [5]:
import numpy as np
import os
import pickle
import pandas as pd
import shutil
from pathlib import Path
from natsort import natsorted

In [6]:
npy_root = Path(f'/content/drive/.shortcut-targets-by-id/1a7cCN6IAhkm0gj53vnNz_41b1ca-FH6C/Thesis Folder/Code Implementation/Keypoint to GCN/Dataset/RWTH-PHOENIX-Weather/HRNet_Outputs_16_frame_div/train/npy_16_frame/')
npy_files = os.listdir(npy_root)
npy_files.sort()

In [7]:
len(npy_files)

7096

In [8]:
my_var = 1000
npy_files_var = npy_files[my_var:my_var+500]

In [9]:
len(npy_files_var)

500

In [10]:
set_var = 'train'
# set_var = 'test'
# set_var = 'dev'

In [11]:
stream = 'joint'
# stream = 'joint_motion'
# stream = 'bone'
# stream = 'bone_motion'

In [13]:
for temp_npy_file in npy_files_var:
    arg = Namespace(
        Experiment_name='Phoenix_test_trial_ver_8', 
        base_lr=0.01, 
        batch_size=64, 
        config='config/sign/test/test_joint_trial.yaml', 
        device=[0], 
        eval_interval=5, 
        feeder='feeders.feeder.Feeder', 
        groups=8, 
        ignore_weights=[], 
        keep_rate=0.9, 
        log_interval=100, 
        model='model.decouple_gcn_attn.Model', 
        model_args={
            'np_name': temp_npy_file,
            'np_save_dir': '/content/drive/.shortcut-targets-by-id/1a7cCN6IAhkm0gj53vnNz_41b1ca-FH6C/Thesis Folder/Code Implementation/Keypoint to GCN/Dataset/RWTH-PHOENIX-Weather/HRNet_Outputs_16_frame_div/{}/sl_gcn_output/{}/'.format(set_var, stream),
            'num_class': 226, 
            'num_point': 27, 
            'num_person': 1, 
            'graph': 'graph.sign_27.Graph', 
            'groups': 16, 
            'block_size': 41, 
            'graph_args': {
                'labeling_mode': 'spatial'
            }
        }, 
        model_saved_name='', 
        nesterov=True, 
        num_epoch=100, 
        num_worker=32, 
        only_train_epoch=1, 
        only_train_part=True, 
        optimizer='SGD', 
        phase='test', 
        print_log=True, 
        save_interval=2, 
        save_score=False, 
        seed=1, 
        show_topk=[1, 5], 
        start_epoch=0, 
        step=[50], 
        test_batch_size=64, 
        test_feeder_args={
            'data_path': '/content/drive/.shortcut-targets-by-id/1a7cCN6IAhkm0gj53vnNz_41b1ca-FH6C/Thesis Folder/Code Implementation/Keypoint to GCN/Dataset/RWTH-PHOENIX-Weather/HRNet_Outputs_16_frame_div/{}/joint/{}_data_{}.npy'.format(set_var, temp_npy_file, stream), 
            'label_path': '/content/drive/.shortcut-targets-by-id/1a7cCN6IAhkm0gj53vnNz_41b1ca-FH6C/Thesis Folder/Code Implementation/Keypoint to GCN/Dataset/RWTH-PHOENIX-Weather/HRNet_Outputs_16_frame_div/{}/pkl/{}_label.pkl'.format(set_var, temp_npy_file), 
            'random_mirror': False, 
            'normalization': True, 
            'debug': False
        }, 
        train_feeder_args={
            'data_path': './data/sign/27_2/train_val_data_joint.npy', 
            'label_path': './data/sign/27_2/train_val_labels.pkl', 
            'debug': False, 
            'random_choose': True, 
            'window_size': 100, 
            'random_shift': True, 
            'normalization': True, 
            'random_mirror': True, 
            'random_mirror_p': 0.5, 
            'is_vector': False
        }, 
        warm_up_epoch=0, 
        weight_decay=0.0001, 
        weights='/content/drive/.shortcut-targets-by-id/1a7cCN6IAhkm0gj53vnNz_41b1ca-FH6C/Thesis Folder/Code Implementation/Keypoint to GCN/CVPR21Chal-SLR/SL-GCN/final_models/27_2/joint_epoch_226_9468.pt', 
        work_dir='/content/drive/.shortcut-targets-by-id/1a7cCN6IAhkm0gj53vnNz_41b1ca-FH6C/Thesis Folder/Code Implementation/Keypoint to GCN/CVPR21Chal-SLR/SL-GCN/work_dir/Personal_use/'
    )

    %cd /content/drive/.shortcut-targets-by-id/1a7cCN6IAhkm0gj53vnNz_41b1ca-FH6C/Thesis Folder/Code Implementation/Keypoint to GCN/CVPR21Chal-SLR/SL-GCN
    init_seed(0)
    processor = Processor(arg)
    processor.start()
    print('___________________DONE__________________________________')
            

/content/drive/.shortcut-targets-by-id/1a7cCN6IAhkm0gj53vnNz_41b1ca-FH6C/Thesis Folder/Code Implementation/Keypoint to GCN/CVPR21Chal-SLR/SL-GCN
Attention Enabled!
Attention Enabled!
Attention Enabled!
Attention Enabled!
Attention Enabled!
Attention Enabled!
Attention Enabled!
Attention Enabled!


/content/drive/.shortcut-targets-by-id/1a7cCN6IAhkm0gj53vnNz_41b1ca-FH6C/Thesis Folder/Code Implementation/Keypoint to GCN/CVPR21Chal-SLR/SL-GCN/model/decouple_gcn_attn.py:29: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  nn.init.kaiming_normal(conv.weight, mode='fan_out')
/content/drive/.shortcut-targets-by-id/1a7cCN6IAhkm0gj53vnNz_41b1ca-FH6C/Thesis Folder/Code Implementation/Keypoint to GCN/CVPR21Chal-SLR/SL-GCN/model/decouple_gcn_attn.py:30: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.
  nn.init.constant(conv.bias, 0)
/content/drive/.shortcut-targets-by-id/1a7cCN6IAhkm0gj53vnNz_41b1ca-FH6C/Thesis Folder/Code Implementation/Keypoint to GCN/CVPR21Chal-SLR/SL-GCN/model/decouple_gcn_attn.py:34: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.
  nn.init.constant(bn.weight, scale)
/content/drive/.shortcut-targets-by-id/1a7cCN6IAhkm0gj53vnNz_41b1ca-FH6C/Thesis Folder/Code Impleme

Attention Enabled!
Attention Enabled!
[ Sun Dec 19 15:27:41 2021 ] Load weights from /content/drive/.shortcut-targets-by-id/1a7cCN6IAhkm0gj53vnNz_41b1ca-FH6C/Thesis Folder/Code Implementation/Keypoint to GCN/CVPR21Chal-SLR/SL-GCN/final_models/27_2/joint_epoch_226_9468.pt.


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 32 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


10
[ Sun Dec 19 15:27:41 2021 ] Model:   model.decouple_gcn_attn.Model.
[ Sun Dec 19 15:27:41 2021 ] Weights: /content/drive/.shortcut-targets-by-id/1a7cCN6IAhkm0gj53vnNz_41b1ca-FH6C/Thesis Folder/Code Implementation/Keypoint to GCN/CVPR21Chal-SLR/SL-GCN/final_models/27_2/joint_epoch_226_9468.pt.
[ Sun Dec 19 15:27:41 2021 ] Eval epoch: 1


...

Eval Accuracy:  0.0  model:  /content/drive/.shortcut-targets-by-id/1a7cCN6IAhkm0gj53vnNz_41b1ca-FH6C/Thesis Folder/Code Implementation/Keypoint to GCN/Dataset/weights_HRNet_trial/Joint_test/Phoenix_test_trial_ver_8
[ Sun Dec 19 16:01:52 2021 ] 	Mean test loss of 1 batches: 12.230976104736328.
[ Sun Dec 19 16:01:52 2021 ] 	Top1: 0.00%
[ Sun Dec 19 16:01:52 2021 ] 	Top5: 0.00%
[ Sun Dec 19 16:01:52 2021 ] Done.

___________________DONE__________________________________
/content/drive/.shortcut-targets-by-id/1a7cCN6IAhkm0gj53vnNz_41b1ca-FH6C/Thesis Folder/Code Implementation/Keypoint to GCN/CVPR21Chal-SLR/SL-GCN
Attention Enabled!
Attention Enabled!
Attention Enabled!
Attention Enabled!
Attention Enabled!
Attention Enabled!
Attention Enabled!
Attention Enabled!
Attention Enabled!
Attention Enabled!
[ Sun Dec 19 16:01:52 2021 ] Load weights from /content/drive/.shortcut-targets-by-id/1a7cCN6IAhkm0gj53vnNz_41b1ca-FH6C/Thesis Folder/Code Implementation/Keypoint to GCN/CVPR21Chal-SLR/SL-GCN/

100%|██████████| 1/1 [00:04<00:00,  4.43s/it]

06October_2009_Tuesday_tagesschau-8161
/content/drive/.shortcut-targets-by-id/1a7cCN6IAhkm0gj53vnNz_41b1ca-FH6C/Thesis Folder/Code Implementation/Keypoint to GCN/Dataset/RWTH-PHOENIX-Weather/HRNet_Outputs_16_frame_div/train/sl_gcn_output/joint/
Size Before fc layer
torch.Size([8, 256])
torch.float32
Groups_______________________


100%|██████████| 1/1 [00:05<00:00,  5.34s/it]

Eval Accuracy:  0.0  model:  /content/drive/.shortcut-targets-by-id/1a7cCN6IAhkm0gj53vnNz_41b1ca-FH6C/Thesis Folder/Code Implementation/Keypoint to GCN/Dataset/weights_HRNet_trial/Joint_test/Phoenix_test_trial_ver_8
[ Sun Dec 19 16:01:58 2021 ] 	Mean test loss of 1 batches: 9.455411911010742.
[ Sun Dec 19 16:01:58 2021 ] 	Top1: 0.00%
[ Sun Dec 19 16:01:58 2021 ] 	Top5: 0.00%
[ Sun Dec 19 16:01:58 2021 ] Done.

___________________DONE__________________________________
